In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms

In [2]:
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.softmax(self.fc4(x), dim=1)
        return x
        

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
suffix = "lr0.01_ep2"

writer = SummaryWriter(log_dir=None, comment=suffix)
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)
trainset = datasets.MNIST("mnist_train", train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
images, labels = next(iter(trainloader))

grid = torchvision.utils.make_grid(images)
writer.add_image("images", grid, 0)

net = SimpleNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
epoch_no = 2

update = 0
for epoch in range(epoch_no):  # loop over the dataset multiple times
    print(epoch)
    for i, data in enumerate(trainloader, 0):
        update += 1
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs = inputs.reshape(-1, 28*28)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        
        writer.add_scalar('Loss/train', loss.to("cpu"), update)
        writer.add_scalar('Acc/train', ((outputs.argmax(1) == labels).sum() / float(len(labels))).to("cpu"), update)
        
        loss.backward()
        optimizer.step()
        
writer.add_graph(net.to("cpu"), images.reshape(-1, 28*28))
writer.close()

0
1
